In [57]:
%load_ext autoreload
%autoreload 2
%run ./initializer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Path (/Users/rik/Documents/VU/DMT/DataMiningTechniquesA2) already exists in sys.path


In [66]:
import pandas as pd
from lgbm_ranker import LGBMRankerModel, LGBMRanker
from feature_tester import FeatureTester
import numpy as np

In [59]:
train_df = pd.read_parquet("data/training_set_processed.parquet")
test_df = pd.read_parquet("data/test_set_processed.parquet")

In [67]:
lgbm_class_model = LGBMRankerModel(
    df = train_df,              # Only initialized to get column names
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[5],
    n_estimators=1000,
    learning_rate=0.05,
)


In [ ]:
X_train, y_train, X_val, y_val, groups_size_train, groups_size_val = lgbm_class_model.format_data(train_df)

In [61]:
fitted_model = lgbm_class_model.fit(X_train, y_train, X_val, y_val, groups_size_train, groups_size_val)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.338295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14118
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.370262
[40]	valid_0's ndcg@5: 0.378783
[60]	valid_0's ndcg@5: 0.381895
[80]	valid_0's ndcg@5: 0.384895
[100]	valid_0's ndcg@5: 0.38763
[120]	valid_0's ndcg@5: 0.389607
[140]	valid_0's ndcg@5: 0.390542
[160]	valid_0's ndcg@5: 0.391963
[180]	valid_0's ndcg@5: 0.392818
[200]	valid_0's ndcg@5: 0.393664
[220]	valid_0's ndcg@5: 0.

In [55]:
lgbm_class_model.save_final_results(test_df)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Final results saved to final_predictions
         srch_id  prop_id
23             1    99484
9              1    54937
12             1    61934
5              1    28181
1              1     5543
...          ...      ...
4959177   332787    29018
4959182   332787    99509
4959181   332787    94437
4959178   332787    32019
4959180   332787    35240

[4959183 rows x 2 columns]


In [37]:
lgbm_class_model.get_final_predictions(test_df)

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


,srch_id,prop_id,predictions
23,1,99484,0.726405
9,1,54937,0.700577
1,1,5543,0.672712
12,1,61934,0.668842
0,1,3180,0.652173
...,...,...,...
4959177,332787,29018,0.223181
4959182,332787,99509,0.127097
4959181,332787,94437,0.008470
4959178,332787,32019,-0.144257


In [70]:
impute_null_columns = {"visitor_hist_starrating": "median", "visitor_hist_adr_usd": "mean", "prop_review_score": "median", "orig_destination_distance": "mean", "prop_log_historical_price":"mean"}
feature_tester = FeatureTester(lgbm_class_model, impute_null_columns=impute_null_columns)

In [71]:
model_opt, model_opt_not_imputed = feature_tester.test_na_fill_method()

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWa

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.346574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14074
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.368835
[40]	valid_0's ndcg@5: 0.375622
[60]	valid_0's ndcg@5: 0.380283
[80]	valid_0's ndcg@5: 0.382256
[100]	valid_0's ndcg@5: 0.385042
[120]	valid_0's ndcg@5: 0.386149
[140]	valid_0's ndcg@5: 0.387534
[160]	valid_0's ndcg@5: 0.38893
[180]	valid_0's ndcg@5: 0.389805
[200]	valid_0's ndcg@5: 0.391009
[220]	valid_0's ndcg@5: 0.

/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pandas/core/dtypes/astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
/Users/rik/.pyenv/versions/3.10.12/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWa

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.382974 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14119
[LightGBM] [Info] Number of data points in the train set: 3966677, number of used features: 110
Training until validation scores don't improve for 50 rounds
[20]	valid_0's ndcg@5: 0.370262
[40]	valid_0's ndcg@5: 0.37878
[60]	valid_0's ndcg@5: 0.382431
[80]	valid_0's ndcg@5: 0.385289
[100]	valid_0's ndcg@5: 0.38792
[120]	valid_0's ndcg@5: 0.389211
[140]	valid_0's ndcg@5: 0.390668
[160]	valid_0's ndcg@5: 0.391901
[180]	valid_0's ndcg@5: 0.393104
[200]	valid_0's ndcg@5: 0.394036
[220]	valid_0's ndcg@5: 0.3